In [1]:
import ray
from model import *

/home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [2]:
ray.init()

2021-05-27 22:24:21,036	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.22',
 'raylet_ip_address': '192.168.0.22',
 'redis_address': '192.168.0.22:45395',
 'object_store_address': '/tmp/ray/session_2021-05-27_22-24-20_040044_207850/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-27_22-24-20_040044_207850/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-05-27_22-24-20_040044_207850',
 'metrics_export_port': 43921,
 'node_id': '369a08fec2f56e34e6140f3a8f50f597ea6cf1c778616eb0d0259b59'}

In [16]:
@ray.remote(num_gpus=1)
class Learner:
    def __init__(self, network):
        self.learner_network = DQN(19).cuda()
        self.learner_network.load_state_dict(network.state_dict())
        
    def get_network(self):
        return self.learner_network.cpu()


In [4]:
a = []
b = [1]

In [5]:
for i, j in zip(a, b):
    print(i, j)

In [28]:
@ray.remote
class Actor:
    def __init__(self, learner):
        self.actor_network = DQN(19).cpu()
        self.actor_target_network = DQN(19).cpu()
        self.shared_network = ray.get(learner.get_network.remote())
        self.actor_network.load_state_dict(self.shared_network.state_dict())
        self.actor_target_network.load_state_dict(self.actor_network.state_dict())

In [18]:
import os

In [19]:
network = DQN(19)
model_path = os.curdir +'/dqn_model/' + 'pre_trained4.pth'
network.load_state_dict(torch.load(model_path, map_location='cuda:0'))

<All keys matched successfully>

In [20]:
learner = Learner.remote(network)

In [29]:
actor = Actor.remote(learner)

2021-05-27 22:34:04,688	WARNING worker.py:1115 -- The autoscaler failed with the following error:
Terminated with signal 15
  File "/home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/ray/_private/monitor.py", line 376, in <module>
    monitor.run()
  File "/home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/ray/_private/monitor.py", line 284, in run
    self._run()
  File "/home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/ray/_private/monitor.py", line 202, in _run
    time.sleep(AUTOSCALER_UPDATE_INTERVAL_S)

2021-05-27 22:34:04,755	ERROR import_thread.py:88 -- ImportThread: Connection closed by server.
2021-05-27 22:34:04,756	ERROR worker.py:922 -- print_logs: Connection closed by server.
2021-05-27 22:34:04,762	ERROR worker.py:1117 -- listen_error_messages_raylet: Connection closed by server.
